In [10]:
from torch.utils.data import DataLoader
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import numpy as np
import time
import os
import datetime
import torch.nn as nn
from torch import autograd

from dataset import PointNetDataset
from model import PointNet

SEED = 13
batch_size = 8
epochs = 100
decay_lr_factor = 0.95
decay_lr_every = 2
lr = 0.01
gpus = [0]
global_step = 0
show_every = 1
val_every = 3
date = datetime.date.today()
save_dir = "../output"


def save_ckp(ckp_dir, model, optimizer, epoch, best_acc, date):
    os.makedirs(ckp_dir, exist_ok=True)
    state = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    ckp_path = os.path.join(
        ckp_dir, f'date_{date}-epoch_{epoch}-maxacc_{best_acc:.3f}.pth')
    torch.save(state, ckp_path)
    torch.save(state, os.path.join(ckp_dir, f'latest.pth'))
    print('model saved to %s' % ckp_path)


def load_ckp(ckp_path, model, optimizer):
    state = torch.load(ckp_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print("model load from %s" % ckp_path)


def softXEnt(prediction, real_class):
    # TODO: return loss here
    cross_entropy_loss = nn.CrossEntropyLoss()
    softmax = np.argmax(real_class, axis = 0 )
    print(softmax)
    target = autograd.Variable(torch.LongTensor(softmax))
    loss = cross_entropy_loss(prediction, target)  # 交叉熵
    return loss


def get_eval_acc_results(model, data_loader, device):
    """
    ACC
    """
    seq_id = 0
    model.eval()

    distribution = np.zeros([5])
    confusion_matrix = np.zeros([5, 5])
    pred_ys = []
    gt_ys = []
    with torch.no_grad():
        accs = []
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)

            # TODO: put x into network and get out
            out = model(x)

            # TODO: get pred_y from out
            pred_y = torch.argmax(nn.Softmax(out))
            gt = np.argmax(y.cpu().numpy(), axis=1)

            # TODO: calculate acc from pred_y and gt
            acc = np.sum(pred_y == gt)
            gt_ys = np.append(gt_ys, gt)
            pred_ys = np.append(pred_ys, pred_y)
            idx = gt

            accs.append(acc)

        return np.mean(accs)

In [2]:
writer = SummaryWriter('./output/runs/tersorboard')
torch.manual_seed(SEED)
device = torch.device(
    f'cuda:{gpus[0]}' if torch.cuda.is_available() else 'cpu')
print("Loading train dataset...")
train_data = PointNetDataset(
    "../../modelnet40_normal_resampled", train=0)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
print("Loading valid dataset...")
val_data = PointNetDataset(
    "../../modelnet40_normal_resampled/", train=1)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
print("Set model and optimizer...")
model = PointNet().to(device=device)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(
    optimizer, step_size=decay_lr_every, gamma=decay_lr_factor)

Loading train dataset...
There are 9843 trian files.
Loading valid dataset...
There are 2468 test files.
Set model and optimizer...


In [16]:
best_acc = 0.0
model.train()
print("Start trainning...")
for epoch in range(epochs):
    acc_loss = 0.0
    num_samples = 0
    start_tic = time.time()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        # TODO: set grad to zero
        optimizer.zero_grad()
        # TODO: put x into network and get out
        with torch.no_grad():
            out = model(x)
        #loss = softXEnt(out, y)
        # TODO: loss backward
        #loss.backward()
        # TODO: update network's param
       # optimizer.step()

Start trainning...


RuntimeError: CUDA out of memory. Tried to allocate 40.00 MiB (GPU 0; 8.00 GiB total capacity; 5.95 GiB already allocated; 11.78 MiB free; 5.99 GiB reserved in total by PyTorch)